In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("synysterjeet/food-classification")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\carla\.cache\kagglehub\datasets\synysterjeet\food-classification\versions\6


In [5]:
import os #path to files
import pickle # save the model
from skimage.io import imread # read image from sklearn
from skimage.transform import resize #resize image
import numpy as np
from sklearn.model_selection import train_test_split #divide files train/test
from sklearn.model_selection import GridSearchCV #choose the best model with best params
from sklearn.svm import SVC #classifier used
from sklearn.metrics import accuracy_score # accuracy for best model


C:\Users\carla\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [9]:
import os
input_dir = r'C:\Users\carla\.cache\kagglehub\datasets\synysterjeet\food-classification\versions\6/dataset/train'
categories =[]
for file in os.listdir(input_dir):
    categories.append(file)
data= []
labels = []
for category_idx,category in enumerate(categories):
     i = 0
     for file in os.listdir(os.path.join(input_dir,category)):
         img_path = os.path.join(input_dir,category,file)
         img = imread(img_path)
         # Check if image has an alpha channel and remove it if present
         if img.shape[-1] == 4:
            img = img[:, :, :3]  # Drop alpha channel

         if img.ndim == 2:  # If grayscale
            img = np.stack([img]*3, axis=-1)  # Convert to RGB-like

         img = resize(img, (15, 15, 3))
         data.append(img.flatten())
         labels.append(category_idx)
         # i+=1
         # if i>50:
         #  break
     print(category_idx)
data = np.asarray(data)
labels = np.asarray(labels)
import numpy as np

np.save("processed_data.npy", data)
np.save("processed_labels.npy", labels)

print("Saved preprocessed data and labels.")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208


C:\Users\carla\anaconda3\Lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (121853,) + inhomogeneous part.

In [15]:
lengths = [d.shape[0] for d in data]
unique_lengths = set(lengths)
print(f"Unique flattened image lengths: {unique_lengths}")


Unique flattened image lengths: {2025, 675}


In [25]:
clean_data = []
clean_labels = []
for img, label in zip(data, labels):
    if img.shape[0] == 675:
        clean_data.append(img)
        clean_labels.append(label)
    else:
        print(f"Skipping image with shape: {img.shape}")

data = np.array(clean_data)
labels = np.array(clean_labels)


Skipping image with shape: (2025,)
Skipping image with shape: (2025,)
Skipping image with shape: (2025,)


In [27]:
print(len(data))

121850


In [29]:
np.save("processed_data.npy", data)
np.save("processed_labels.npy", labels)
print("Saved cleaned preprocessed data and labels.")


Saved cleaned preprocessed data and labels.


In [17]:
import pickle

with open("raw_data.pkl", "wb") as f:
    pickle.dump(data, f)

with open("raw_labels.pkl", "wb") as f:
    pickle.dump(labels, f)

print("Saved raw (possibly inconsistent) data and labels with pickle.")


Saved raw (possibly inconsistent) data and labels with pickle.


In [19]:
with open("raw_data.pkl", "rb") as f:
    loaded_data = pickle.load(f)

with open("raw_labels.pkl", "rb") as f:
    loaded_labels = pickle.load(f)


121853

In [31]:
x_train, x_test, y_train,y_test = train_test_split(data,labels,test_size = 0.2,shuffle = True,stratify = labels)

In [33]:
classifier = SVC()
parameters = [{'gamma':[0.01],'C':[100]}]
grid_search = GridSearchCV(classifier, parameters)
grid_search.fit(x_train,y_train)

GridSearchCV(estimator=SVC(), param_grid=[{'C': [100], 'gamma': [0.01]}])

In [34]:
best_estimator = grid_search.best_estimator_
y_prediction = best_estimator.predict(x_test)
score = accuracy_score(y_prediction, y_test)
print(f'{score*100}% of samples were correctly classified')

84.661469019286% of samples were correctly classified


In [42]:
pickle.dump(best_estimator,open('./490Image.pkl','wb'))

In [7]:
import os
import numpy as np
input_dir = r'C:\Users\carla\.cache\kagglehub\datasets\synysterjeet\food-classification\versions\6/dataset/train'
categories = sorted(os.listdir(input_dir))  # sort to ensure consistent order
category_dict = {idx: category for idx, category in enumerate(categories)}
np.save('category_dict.npy', category_dict)